###### Here I'm using findspark package to indicate where spark installed.

In [1]:
import findspark
findspark.init(r'C:\Users\q1011812\Downloads\spark-3.0.1-bin-hadoop3.2\spark-3.0.1-bin-hadoop3.2')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('clust-project').getOrCreate()

In [3]:
dataset = spark.read.csv('hack_data.csv', 
                        header = True,
                        inferSchema = True)
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [4]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [5]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used','Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed'], outputCol = 'features')
dataset = assembler.transform(dataset)

In [6]:
dataset = dataset.select('features')

In [7]:
from pyspark.ml.feature import StandardScaler
sc = StandardScaler(inputCol = 'features',
                   outputCol = 'scaledFeatures')
scaled_model = sc.fit(dataset)

In [8]:
dataset = scaled_model.transform(dataset)

In [9]:
dataset.select('scaledFeatures').show(5)

+--------------------+
|      scaledFeatures|
+--------------------+
|[0.56785108466505...|
|[1.41962771166263...|
|[2.20042295307707...|
|[0.14196277116626...|
|[1.41962771166263...|
+--------------------+
only showing top 5 rows



In [10]:
kmeans3 = KMeans(featuresCol = 'scaledFeatures').setK(3).setSeed(1)
kmeans2 = KMeans(featuresCol = 'scaledFeatures').setK(2).setSeed(1)

In [11]:
model3 = kmeans3.fit(dataset)
model2 = kmeans2.fit(dataset)

In [12]:
model3.transform(dataset).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   79|
|         0|   88|
+----------+-----+



In [13]:
model2.transform(dataset).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



### 2 clusters